In [31]:
import pandas as pd
import numpy as np
# %run "A3_tools.py"
hourly_bear_data = pd.read_pickle('hourly_bear.pkl')
minute_bull_data = pd.read_pickle('minutely_bull.pkl')
hourly_bull_data = pd.read_pickle('hourly_bull.pkl')
minute_crab_data = pd.read_pickle('minutely_crab.pkl')

#No trading fees (eToro, Robinhood, Shakepay, Binance Spot (BTCUSD))

In [32]:
# Parameters
initial_investment = 1
# Hold
hold_return = {
  'name': 'hold_return',
  'hourly_bear': (float(hourly_bear_data.tail(1)['close']) - float(hourly_bear_data.head(1)['close'])) / float(hourly_bear_data.head(1)['close']) * initial_investment,
  'hourly_bull': (float(hourly_bull_data.tail(1)['close']) - float(hourly_bull_data.head(1)['close'])) / float(hourly_bull_data.head(1)['close']) * initial_investment,
  'minutely_crab': (float(minute_crab_data.tail(1)['close']) - float(minute_crab_data.head(1)['close'])) / float(minute_crab_data.head(1)['close']) * initial_investment,
  'minutely_bull': (float(minute_bull_data.tail(1)['close']) - float(minute_bull_data.head(1)['close'])) / float(minute_bull_data.head(1)['close']) * initial_investment,
}

# hold_mdd = {
#   'name': 'hold_mdd',
#   'hourly_bear': 
# }

In [33]:
def mdd(df):
  max_price = 0
  max_drawdown = 0
  for index, row in df.iterrows():
    if row['high'] > max_price:
      max_price = row['high']
    if (max_price - row['close']) / max_price > max_drawdown:
      max_drawdown = (max_price - row['close']) / max_price
    df.at[index, 'mdd'] = max_drawdown
  return max_price, max_drawdown

#100-period sortino ratio
def roll_sortino(df):
  risk_free = 0 #0 percent
  return_negative = df.to_numpy()
  return_negative_std = return_negative[return_negative < 0].std()
  return (df.mean() - risk_free) / return_negative_std * np.sqrt(100)

def sortino(df, time_period): #time period: m for minute, h for hour
  df['sortino'] = df['return'].rolling('100{0}'.format(time_period)).apply(roll_sortino)

def sortino_average(df):
  return df['sortino'].to_numpy()[100:].mean()

In [34]:
hold_return

{'name': 'hold_return',
 'hourly_bear': -0.4484251546191791,
 'hourly_bull': 0.2643392448215483,
 'minutely_crab': -0.0361619993414265,
 'minutely_bull': 0.044083312175036145}

In [35]:
max_drawdown = {
  'name': 'max_drawdown',
  'hourly_bear': mdd(hourly_bear_data)[1] / mdd(hourly_bear_data)[0],
  'hourly_bull': mdd(hourly_bull_data)[1] / mdd(hourly_bull_data)[0],
  'minutely_crab': mdd(minute_crab_data)[1] / mdd(minute_crab_data)[0],
  'minutely_bull': mdd(minute_bull_data)[1] / mdd(minute_bull_data)[0],
}

In [36]:
max_drawdown

{'name': 'max_drawdown',
 'hourly_bear': 7.537888225836388e-06,
 'hourly_bull': 6.365487109731123e-06,
 'minutely_crab': 2.8755547992800673e-06,
 'minutely_bull': 7.371656698379641e-07}

In [37]:
for i in [(hourly_bear_data, 'h'), (hourly_bull_data, 'h'), (minute_crab_data, 'T'), (minute_bull_data, 'T')]:
  sortino(i[0], i[1])

sortino_ratio = {
  'name': 'sortino_ratio',
  'hourly_bear': sortino_average(hourly_bear_data),
  'hourly_bull': sortino_average(hourly_bull_data),
  'minutely_crab': sortino_average(minute_crab_data),
  'minutely_bull': sortino_average(minute_bull_data)
}

c:\users\clementich\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice



In [38]:
sortino_ratio

{'name': 'sortino_ratio',
 'hourly_bear': -0.398236991472321,
 'hourly_bull': 0.7060084215446735,
 'minutely_crab': -0.22755495976134527,
 'minutely_bull': 1.0024430555166253}

In [39]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [40]:
def plot_fig(df, title, custom_range=[0,1]):
  # Price line
  fig = make_subplots(rows=2, cols=1, 
                      specs = [[{"secondary_y": True}], 
                              [{"secondary_y": False}]])
  fig.update_layout(
      autosize=False,
      width=1300,
      height=800,
      title_text=title,
      yaxis_range=custom_range,
      yaxis2_range=[custom_range[0] - 1, custom_range[1] - 1]
    )

  initial_btc_price = float(df.head(1)['close'])
  print(initial_btc_price)
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['close'] / initial_btc_price,
          # line=dict(color='#ff9900', width=1),
          name='BTC = Portfolio',
          # showlegend=False,
          legendgroup='1',
          marker=dict(
          size=42,
          # I want the color to be green if 
          # lower_limit ≤ y ≤ upper_limit
          # else red
          color='black',
        )
      ), row=1, col=1
  )

  # Sortino
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['sortino'],
          name='Sortino',
      ), row=2, col=1
  )

  # Portfolio
  fig.add_trace(
    go.Scatter(
          x=df.index,
          y=(df['close']) / initial_btc_price,
          # line=dict(color='#ff9900', width=1),
          name='Portfolio',
          # showlegend=False,
          marker=dict(
          size=42,
          # I want the color to be green if 
          # lower_limit ≤ y ≤ upper_limit
          # else red
          color='red',
        )
      ), row=1, col=1
  )

  # MDD
  fig.add_trace(
    go.Scatter(
          x=df.index,
          y=-df['mdd'],
          # line=dict(color='#ff9900', width=1),
          name='MDD',
          # showlegend=False,
          marker=dict(
          size=42,
          # I want the color to be green if 
          # lower_limit ≤ y ≤ upper_limit
          # else red
          color='blue',
        )
      ), secondary_y=True, row=1, col=1
  )
  return fig

In [51]:
fig_hr_bear = plot_fig( hourly_bear_data, "Hourly Bear Data", [0.45,1.05])
fig_hr_bull = plot_fig( hourly_bull_data, "Hourly Bull Data", [0.8,1.35])
fig_mi_crab = plot_fig( minute_crab_data, "Minute Crab Data", [0.9,1.03])
fig_mi_bull = plot_fig( minute_bull_data, "Minute Bull Data", [0.98,1.06])

68054.3
18292.63
24689.73
21740.88


In [52]:
fig_hr_bear

In [53]:
fig_hr_bull

In [54]:
fig_mi_crab

In [55]:
fig_mi_bull